In [2]:
from pathlib import Path
import string
from dataclasses import dataclass
from typing import Self

In [3]:

class Node:
    def __init__(self, name : int, isdir : bool = False, size : int | None = None):
        assert isdir or (size is not None), 'Set either size or isdir=True'
        self.name = name
        self.isdir = isdir
        self._size = int(size) if size is not None else None
        self._children = None

    @classmethod
    def from_str(cls, s : str):
        dir_or_size, name = s.split()
        if dir_or_size == 'dir':
            return cls(name, isdir=True, size=None)
        else:
            return cls(name, isdir=False, size=dir_or_size)

    @property
    def size(self):
        if self._size is not None:
            return self._size
        assert self.isdir
        self._size = sum(c.size for c in self.children)
        return self._size

    def __repr__(self) -> str:
        params = ', '.join([f'name="{self.name}"',
                          f'isdir={self.isdir}',
                          f'size={self._size}'])
        if self.isdir:
            params += f', #children={len(self.children)}'
        return f'Node({params})'

    def walk(self) -> list[Self]:
        nodes = [self]
        if self.isdir and self.children:
            for c in self.children:
                nodes.extend(c.walk())
        return nodes

    def make_dirs(self, *lines):
        nodes = [Node.from_str(o) for o in lines]
        self._children = {node.name : node for node in nodes}

    def get_child(self, child):
        return self._children[child]
    
    @property
    def children(self):
        return self._children.values()

In [4]:
class FS:
    def __init__(self):
        self.root = Node('/', isdir=True, size=None)
        self.path = []

    @property
    def cwd(self):
        return self.path[-1]

    def up(self):
        self.path.pop()

    def down(self, child : str):
        self.path.append(self.cwd.get_child(child))

    def to_root(self):
        self.path = [self.root]


    def process(self, line : str):
        (inp, out) = line.split('\n', 1)
        inp = inp.split()
        out = out.rstrip().split('\n')

        match inp:
            case ('ls',):
                self.cwd.make_dirs(*out)
            case ('cd', '/'):
                self.to_root()
            case ('cd', '..'):
                self.up()
            case ('cd', str as dir_):
                self.down(dir_)
            case _:
                raise Exception(f'Case not found, {inp}')

    def walk(self):
        nodes = [self]

In [5]:
def load_lines():
    lines = Path(f'data/07.txt').read_text().rstrip().split('$')
    lines = [l for l in lines if len(l) > 0]
    return lines


In [6]:
lines = load_lines()
fs = FS()
for line in lines:
    fs.process(line)

In [7]:
fs.root.walk()

[Node(name="/", isdir=True, size=None, #children=11),
 Node(name="brdsppd", isdir=True, size=None, #children=6),
 Node(name="gjc", isdir=True, size=None, #children=6),
 Node(name="bvctghh", isdir=True, size=None, #children=3),
 Node(name="lsprzlbf", isdir=True, size=None, #children=6),
 Node(name="hhfqgzfj.qvt", isdir=False, size=182522),
 Node(name="hts", isdir=True, size=None, #children=2),
 Node(name="dnltsq.fwv", isdir=False, size=11158),
 Node(name="tchv", isdir=False, size=52582),
 Node(name="jtpdh", isdir=False, size=229288),
 Node(name="lwfgnzz", isdir=True, size=None, #children=1),
 Node(name="tjslbpb", isdir=True, size=None, #children=1),
 Node(name="jtzmjgw.bql", isdir=False, size=58586),
 Node(name="szfw", isdir=False, size=284594),
 Node(name="tgdsjl", isdir=False, size=89639),
 Node(name="lwfgnzz", isdir=True, size=None, #children=1),
 Node(name="rlhz.pbs", isdir=False, size=199598),
 Node(name="tjslbpb", isdir=True, size=None, #children=1),
 Node(name="fmzfs.glg", isdir=

In [8]:
pt1 = sum(n.size for n in fs.root.walk() if n.isdir and n.size <= 100000)
pt1

1644735

In [9]:
total_space = 70000000
install_size = 30000000
used_space = fs.root.size
unused_space = total_space - used_space
space_to_free = install_size - unused_space
space_to_free


1272621

In [10]:
pt2 = min(n.size for n in fs.root.walk() if n.isdir and n.size >= space_to_free)
pt2

1300850